# Fetch data

In [ ]:
from ElexonDataPortal import api

your_api_key = ""

client = api.Client(your_api_key)

In [3]:
# ~ 3 months of historical price data

start_date = '2021-08-01'
end_date = '2021-11-14'


df = client.get_DETSYSPRICES(start_date, end_date)

# save the DF
df.to_csv("prices.csv")

df2 = client.get_DEVINDOD(start_date, end_date)
df2.to_csv("demand.csv")

# Clean data and average

In [20]:
import pandas as pd
import datetime

# load the dataframes
prices = pd.read_csv("data/prices.csv")

prices = prices[prices["recordType"] == "MAIN PRICE SUMMARY"]
prices = prices[["local_datetime", "value"]]

prices.to_excel("data/prices_filtered.xlsx")

prices["local_datetime"] = pd.to_datetime(prices["local_datetime"])
prices = prices[prices["local_datetime"] <= datetime.datetime(2021, 10, 26, 0, 0, 0, 0, tzinfo=datetime.timezone.utc)]
prices["hour"] = prices["local_datetime"].apply(lambda x: x.hour)
a = prices.groupby(["hour"]).mean()
print(a)
a.to_excel("average energy price from august to oct25.xlsx")

demand = pd.read_csv("data/half-hourly-demand.csv")

demand = demand[demand["SETTLEMENT_PERIOD"]%2 == 1]
demand["hour"] = demand["SETTLEMENT_PERIOD"].apply(lambda x: int(x/2))
a = demand.groupby(["hour"]).mean()

a.to_excel("average energy demand from august to oct25.xlsx")

           value
hour            
0     119.054424
1     111.653833
2     110.653881
3     104.004906
4     117.639172
5     116.294484
6     133.999658
7     148.676970
8     159.222840
9     173.804423
10    168.835216
11    152.680319
12    152.706402
13    148.545899
14    148.182152
15    131.522738
16    184.641363
17    213.218904
18    242.781162
19    189.254591
20    137.512033
21    130.969195
22    123.653921
23    131.656022


## Get August only data

In [9]:
import pandas as pd

# load the dataframes
prices = pd.read_csv("data/prices.csv")
prices = prices[prices["recordType"] == "MAIN PRICE SUMMARY"]
prices = prices[["local_datetime", "value"]]
prices["local_datetime"] = pd.to_datetime(prices["local_datetime"], utc=True)
prices["local_datetime"] = prices["local_datetime"][prices["local_datetime"].dt.month == 8]
prices["hour"]= prices["local_datetime"].apply(lambda x: x.hour)
a = prices.groupby(["hour"]).mean()
a.to_excel("data/average energy price of august.xlsx")


demand = pd.read_csv("data/half-hourly-demand.csv")
demand["local_datetime"] = pd.to_datetime(demand["SETTLEMENT_DATE"], utc=True)
demand["local_datetime"] = demand["local_datetime"][demand["local_datetime"].dt.month == 8]
demand = demand[demand["SETTLEMENT_PERIOD"]%2 == 1]
demand["hour"] = demand["SETTLEMENT_PERIOD"].apply(lambda x: int(x/2))
a = demand.groupby(["hour"]).mean()
a.to_excel("data/average energy demand for august.xlsx")